In [206]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import matplotlib
import numpy as np
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'text.usetex': False,
    'font.size': 6,
    'axes.labelsize': 6,
    'legend.fontsize': 6,
    'xtick.labelsize': 6,
    'ytick.labelsize': 6
})

os.chdir(os.getcwd())



In [207]:
df1 = pd.read_csv("./input_files/input_benchmarks.csv")
df1bis = pd.read_csv("./input_files/input_benchmarks.csv")
df3 = pd.read_csv("./input_files/input_benchmarks3.csv")
df4 = pd.read_csv("./input_files/input_benchmarks4.csv")
df = pd.concat([df1,df3, df4], ignore_index=True)

plot_folder = "plots3"
#os.mkdir(plot_folder)

import seaborn as sns
pal = sns.color_palette("magma", 256)

# Extract the first and last colors (extreme ends)
extreme_colors = [pal[50], pal[-70]]

In [ ]:
cols = ['states', 'transitions', 'chain_states', 'bit_width']

scaler = MinMaxScaler()
df_norm = df.copy()
df_norm[cols] = scaler.fit_transform(df[cols])

weights = {
    'states': 1,
    'transitions': 0.3,
    'chain_states': 0.3,
    'bit_width':1.5
}

df['model_complexity'] = (
    df_norm['states'] * weights['states'] +
    df_norm['transitions'] * weights['transitions'] +
    df_norm['chain_states'] * weights['chain_states'] +
    df_norm['bit_width'] * weights['bit_width']
)



props_cols = ['atomic_props', 'base_props', 'refined_props', 'num_classes']

df_props_norm = df.copy()
df_props_norm[props_cols] = MinMaxScaler().fit_transform(df[props_cols])

weights_props = {
    'atomic_props': 1.,
    'base_props': 1,
    'refined_props':1,
    'num_classes': 1
}
df['property_complexity'] = (
    df_props_norm['atomic_props'] * weights_props['atomic_props'] +
    df_props_norm['base_props'] * weights_props['base_props'] +
    df_props_norm['refined_props'] * weights_props['refined_props'] +
    df_props_norm['num_classes'] * weights_props['num_classes']
)

df['complexity_metric'] = df['model_complexity']/df['property_complexity'] 
original_df = df.copy()


In [209]:
df

,states,transitions,atomic_props,base_props,refined_props,num_classes,chain_states,bit_width,model_complexity,property_complexity,complexity_metric
0,1000,800,40,3,5,10,10,10,1.188158,1.368646,0.868127
1,2000,1600,40,3,5,10,10,10,1.204609,1.368646,0.880147
2,4000,3200,40,3,5,10,15,10,1.276986,1.368646,0.933029
3,8000,6400,40,3,5,10,15,10,1.342792,1.368646,0.981110
4,16000,12800,40,3,5,10,10,10,1.434930,1.368646,1.048431
...,...,...,...,...,...,...,...,...,...,...,...
75,2000,1600,23,5,36,16,5,10,1.165136,1.537787,0.757671
76,16000,12800,40,5,31,29,23,10,1.537562,2.646424,0.580996
77,2000,1600,30,4,6,30,28,12,1.596715,1.449843,1.101302
78,1000,800,31,2,37,30,13,10,1.211842,2.186235,0.554306


In [210]:
#df_norm_sorted = df.sort_values(by='complexity_metric', ascending=False)

In [211]:
common_folders = ["result_syn1","result_syn3", "result_syn_fin"]
common_name = "resultBenchmark"
file_list = []
for common_folder in common_folders:
    file_list += [f"{common_folder}{i}/{common_name}.csv" for i in ["","_001", "_002", "_003"]]


In [212]:
def get_dfslist(file_list):
    
    df_list = []
    for file in file_list:
        try:
            df_list.append(pd.read_csv(file))
        except Exception as e:
            print(f"WARNING:{e} ")
    return df_list


df_list = get_dfslist(file_list)
result_df = pd.concat(df_list,axis=0, ignore_index=True)
df = df.loc[result_df.index]

In [213]:
result_df.to_csv("result_df.csv")

In [ ]:
original_df_combined_df = pd.concat([df, result_df], axis=1)
original_df_combined_df = original_df_combined_df.dropna(subset=["model_complexity", "property_complexity", "diff"])
labels = ["Trivial", "Standard", "Complex","Hard"]
original_df_combined_df["model_complexity_category"] = pd.qcut(original_df_combined_df["model_complexity"], q=len(labels), labels=labels)
original_df_combined_df["property_complexity_category"] = pd.qcut(original_df_combined_df["property_complexity"], q=len(labels), labels=labels)
original_df_combined_df["model_prop_com"] = (
    "Model=" + original_df_combined_df["model_complexity_category"].astype(str)
    + " / Specification=" + original_df_combined_df["property_complexity_category"].astype(str)
)
original_df_combined_df["model_prop_com_small"] = (
    original_df_combined_df["model_complexity_category"].str[0]
    + "/" + original_df_combined_df["property_complexity_category"].str[0]
)

original_df_combined_df["model_prop_com_med"] = (
    "Model= " + original_df_combined_df["model_complexity_category"].str[0]
    + " / Specification= " + original_df_combined_df["property_complexity_category"].str[0]
)

original_df_combined_df["diff_norm"] = scaler.fit_transform(original_df_combined_df[["diff"]])
original_df_combined_df["diff_norm"] = original_df_combined_df[["diff"]]*10
original_df_combined_df["faster"] = original_df_combined_df["diff"]>0



In [215]:
original_df_combined_df[original_df_combined_df["diff"]<-30]

,states,transitions,atomic_props,base_props,refined_props,num_classes,chain_states,bit_width,model_complexity,property_complexity,...,analysis_time,time_mc_refined,diff,model_complexity_category,property_complexity_category,model_prop_com,model_prop_com_small,model_prop_com_med,diff_norm,faster
33,20000,16004,31,3,14,14,5,2,0.461282,1.230994,...,4.652894,0.022667,-45.192039,Trivial,Standard,Model=Trivial / Specification=Standard,T/S,Model= T / Specification= S,-451.920395,False
34,20000,16005,26,4,3,10,4,2,0.453392,0.636077,...,1.295376,0.015110,-48.518625,Trivial,Trivial,Model=Trivial / Specification=Trivial,T/T,Model= T / Specification= T,-485.186251,False
36,20000,16009,39,5,22,26,2,1,0.312621,2.284076,...,28.237088,0.041858,-55.715813,Trivial,Hard,Model=Trivial / Specification=Hard,T/H,Model= T / Specification= H,-557.158130,False
37,20000,16008,24,2,28,12,20,1,0.454722,1.113360,...,26.539869,0.056609,-180.365202,Trivial,Trivial,Model=Trivial / Specification=Trivial,T/T,Model= T / Specification= T,-1803.652019,False
38,80000,64070,30,3,38,15,30,1,1.521053,1.820063,...,27.077876,0.019789,-80.314092,Hard,Complex,Model=Hard / Specification=Complex,H/C,Model= H / Specification= C,-803.140922,False
39,20000,16600,32,5,14,17,14,1,0.410160,1.473684,...,13.344693,0.033302,-56.413188,Trivial,Complex,Model=Trivial / Specification=Complex,T/C,Model= T / Specification= C,-564.131876,False


In [251]:
#combined_df = original_df_combined_df.drop(index=range(33, 40)) # they skew the result too much, but are considered when analyzed
combined_df = original_df_combined_df.drop(index=[36, 37, 38, 39])

In [252]:
combined_df[combined_df["diff"]<-30]

,states,transitions,atomic_props,base_props,refined_props,num_classes,chain_states,bit_width,model_complexity,property_complexity,...,analysis_time,time_mc_refined,diff,model_complexity_category,property_complexity_category,model_prop_com,model_prop_com_small,model_prop_com_med,diff_norm,faster
33,20000,16004,31,3,14,14,5,2,0.461282,1.230994,...,4.652894,0.022667,-45.192039,Trivial,Standard,Model=Trivial / Specification=Standard,T/S,Model= T / Specification= S,-451.920395,False
34,20000,16005,26,4,3,10,4,2,0.453392,0.636077,...,1.295376,0.015110,-48.518625,Trivial,Trivial,Model=Trivial / Specification=Trivial,T/T,Model= T / Specification= T,-485.186251,False


In [253]:
combined_df.to_csv("combined_df.csv")

In [254]:
combined_df[combined_df["diff"]<-60]

,states,transitions,atomic_props,base_props,refined_props,num_classes,chain_states,bit_width,model_complexity,property_complexity,...,analysis_time,time_mc_refined,diff,model_complexity_category,property_complexity_category,model_prop_com,model_prop_com_small,model_prop_com_med,diff_norm,faster


In [ ]:
summary_df = (
    combined_df.groupby("model_prop_com_med")["diff"]
    .agg(estimate="mean", sem="sem")
    .reset_index()
)

summary_df["ll"] = summary_df["estimate"] - 1 * summary_df["sem"]
summary_df["ul"] = summary_df["estimate"] + 1 * summary_df["sem"]

summary_df["color"] = np.where(summary_df["estimate"] >= 0, "green", "red")

In [256]:
summary_df

,model_prop_com_med,estimate,sem,ll,ul,color
0,Model= C / Specification= C,5.252528,3.696260,1.556268,8.948788,green
1,Model= C / Specification= H,4.975003,2.106505,2.868498,7.081508,green
2,Model= C / Specification= S,3.663627,2.069525,1.594102,5.733152,green
3,Model= C / Specification= T,3.413794,1.580068,1.833726,4.993862,green
4,Model= H / Specification= C,9.614442,4.054503,5.559939,13.668944,green
5,Model= H / Specification= H,9.111401,3.597104,5.514297,12.708505,green
6,Model= H / Specification= S,14.514074,10.455442,4.058633,24.969516,green
7,Model= H / Specification= T,8.338996,3.743449,4.595547,12.082445,green
8,Model= S / Specification= C,5.428918,4.868784,0.560135,10.297702,green
9,Model= S / Specification= H,5.276190,2.330663,2.945527,7.606853,green


In [257]:
nbsp = "\u00A0"
#summary_df["Model Property"] = summary_df["model_prop_com_med"].str.extract(
#    r"Model=(\w+)\s*/\s*Property=(\w+)"
#).apply(lambda x: f"{x[0][0].capitalize()} {x[1]}"+ (nbsp*6 ))
summary_df["Model Property"] = summary_df["model_prop_com_med"].apply(lambda x: f"{x}"+ (nbsp*3))
summary_df
summary_df
#
#summary_df["Model Property 2"] = summary_df["model_prop_com_small"].apply(lambda x: f"{x}"+ (nbsp *0))
#
#summary_df["Model Property 3"] = summary_df["model_prop_com_small"].str.extract(
#    r"Model=(\w+)\s*/\s*Property=(\w+)"
#).apply(lambda x: f"Model= {x[0][0].capitalize()}, Property= {x[1][0].capitalize()}         "+ (nbsp), axis=1)
#summary_df

,model_prop_com_med,estimate,sem,ll,ul,color,Model Property
0,Model= C / Specification= C,5.252528,3.696260,1.556268,8.948788,green,Model= C / Specification= C
1,Model= C / Specification= H,4.975003,2.106505,2.868498,7.081508,green,Model= C / Specification= H
2,Model= C / Specification= S,3.663627,2.069525,1.594102,5.733152,green,Model= C / Specification= S
3,Model= C / Specification= T,3.413794,1.580068,1.833726,4.993862,green,Model= C / Specification= T
4,Model= H / Specification= C,9.614442,4.054503,5.559939,13.668944,green,Model= H / Specification= C
5,Model= H / Specification= H,9.111401,3.597104,5.514297,12.708505,green,Model= H / Specification= H
6,Model= H / Specification= S,14.514074,10.455442,4.058633,24.969516,green,Model= H / Specification= S
7,Model= H / Specification= T,8.338996,3.743449,4.595547,12.082445,green,Model= H / Specification= T
8,Model= S / Specification= C,5.428918,4.868784,0.560135,10.297702,green,Model= S / Specification= C
9,Model= S / Specification= H,5.276190,2.330663,2.945527,7.606853,green,Model= S / Specification= H


In [ ]:
import forestplot as fp
import matplotlib.pyplot as plt
import seaborn as sns
pal = sns.color_palette("magma", 256)
sns.set_theme(style="dark")

# Generate forest plot
fig = fp.forestplot(
    summary_df,
    estimate="estimate",
    ll="ll",
    hl="ul",
    varlabel="Model Property",
    color="color",
    xlabel=None,
    xlim=(-31, 30),  
    figsize=(7, 4), 
    sort=True,
    ci_report=False, 
    flush=True,
    color_alt_rows=True,
    #              annote=["est_ci"],  # columns to report on left of plot
    #          annoteheaders=[ "            Est. (95% Conf. Int.)"],  # ^corresponding headers
    table=True, pad_difference = 80,
    **{"marker": "s",  # set maker symbol as diamond
                 "markersize": 13,  # adjust marker size
                 "xlinecolor": extreme_colors[0],  # gray color for x-reference line
                 "xtick_size": 12,  # adjust x-ticker fontsize
                 "markercolor" :extreme_colors[0],
                 "linecolor":extreme_colors[0],
                 "fontsize":14,
                 "xlabel_size":14,
                 "xlabel_fontweight":"normal",
                 "fontfamily" : "serif",
                 "row_color" : "white",
                 "alpha_row_color" : 0.7
                }  
)
plt.tight_layout()
plt.show()
plt.savefig(f"{plot_folder}/forest.png")
plt.savefig(f"{plot_folder}/forest.pgf")
plt.close()

{'color': 'color', 'xlim': (-31, 30), 'pad_difference': 80, 'marker': 's', 'markersize': 13, 'xlinecolor': (0.225302, 0.060445, 0.431742), 'xtick_size': 12, 'markercolor': (0.225302, 0.060445, 0.431742), 'linecolor': (0.225302, 0.060445, 0.431742), 'fontsize': 14, 'xlabel_size': 14, 'xlabel_fontweight': 'normal', 'fontfamily': 'serif', 'row_color': 'white', 'alpha_row_color': 0.7}


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

complexity_norm =  (combined_df['model_complexity'])

plt.figure(figsize=(10,6))

y_cols = ['time_brute_force', 'total_time_with_analysis', 'analysis_time', 'time_mc_refined']

for col in y_cols:
    plt.plot(complexity_norm, combined_df[col], marker='o', label=col)

plt.xlabel('Complexity Metric (normalized 0-100)')
plt.ylabel('Time (seconds)')
plt.title('Time vs Normalized Complexity Metric')
plt.legend()
plt.grid(True)
plt.close()

In [225]:
# Normalize complexity_metric to 0-100 scale
complexity_norm =  (combined_df['model_complexity'])

# Prepare plot
plt.figure(figsize=(10,6))

# List of columns to plot on y-axis
y_cols = ['time_brute_force', 'total_time_with_analysis', 'analysis_time', 'time_mc_refined']
fig, ax = plt.subplots()
ax.scatter(combined_df['model_complexity'], combined_df['property_complexity'], marker='o', label=col)


ax.set_xlabel('Complexity Metric (normalized 0-100)')
ax.set_ylabel('Time (seconds)')
ax.set_title('Time vs Normalized Complexity Metric')
ax.legend()
ax.grid(True)
fig.savefig(f"{plot_folder}/scatter_plot.png")
fig.savefig(f"{plot_folder}/scatter_plot.pgf")
plt.close()

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
fig, ax = plt.subplots(figsize=(5, 3)) 



# use the scatterplot function
sns_plot = sns.scatterplot(
    data=combined_df,
    x="model_complexity",
    y="property_complexity",
    size="analysis_time",
    hue="faster",
    alpha=0.8,
    sizes=(30, 400),
    legend="brief",
    ax=ax,
    palette=extreme_colors,
)
sns_plot.axes.set_xlabel("Model Complexity")
sns_plot.axes.set_ylabel("Specification Complexity")
legend = sns_plot.legend_
sns_plot.axes.legend(loc='upper left')
sns_plot.figure.tight_layout()

handles, labels = sns_plot.get_legend_handles_labels()
sns_plot.legend(
    handles=handles[1:3],# + [handles[4],[handles[5], handles[-5]],
    labels=["Slower", "Faster"],#,  "Difference in time", "Small"," Big"],
    title="Legend", loc='upper left'
)

sns_plot.figure.savefig(f"{plot_folder}/scatter_plot_modl_prop.png")
sns_plot.figure.savefig(f"{plot_folder}/scatter_plot_modl_prop.pgf")
plt.close()

In [227]:
import random

In [229]:
import seaborn as sns
sns.set_theme(style="whitegrid")

# use the scatterplot function
sns_plot = sns.scatterplot(
    data=combined_df,
    x="model_complexity",
    y="diff",
    size="time_brute_force",
    hue="faster",
    alpha=0.5,
    legend=False,
    sizes=(20, 400)
)
sns_plot.figure.savefig("output_pissi.pgf")


sns_plot.figure.savefig(f"{plot_folder}/scatter_plot_modl.png")
sns_plot.figure.savefig(f"{plot_folder}/scatter_plot_modl.pgf")

In [230]:
import seaborn as sns
sns.set_theme(style="darkgrid")

# use the scatterplot function
sns_plot = sns.scatterplot(
    data=combined_df,
    x="property_complexity",
    y="diff",
    size="time_brute_force",
    hue="faster",
    alpha=0.5,
    legend=False,
    sizes=(20, 400)
)
sns_plot.figure.savefig("output_pissi.pgf")
sns_plot.figure.savefig(f"{plot_folder}/scatter_plot_prop.png")
sns_plot.figure.savefig(f"{plot_folder}/scatter_plot_prop.pgf")

In [231]:

fig, ax = plt.subplots(figsize=(5, 3)) 

sns_plot = sns.boxplot(data=combined_df, x="model_prop_com", y="diff", hue="faster", ax=ax, palette=extreme_colors)
sns_plot.axes.set_xlabel("Model and Property complexity")
sns_plot.axes.set_ylabel("Time Difference in percentage")
sns_plot.axes.tick_params(axis='x', rotation=90)
#plt.legend(title="Property Complexity")


legend = sns_plot.legend_
sns_plot.axes.legend(loc='upper left')
sns_plot.figure.tight_layout()

sns_plot.figure.savefig(f"{plot_folder}/boxplot.png")
sns_plot.figure.savefig(f"{plot_folder}/boxplot.pgf")

In [232]:


fig, ax = plt.subplots(figsize=(5, 3)) 
pal = sns.color_palette("magma", 256)


sns_plot = sns.violinplot(data=combined_df, x="model_complexity_category", y="diff", hue="faster", split=True, ax=ax, palette=extreme_colors)
sns_plot.axes.set_xlabel("Model Complexity Category")
sns_plot.axes.set_ylabel("Time Difference in percentage")
sns_plot.axes.tick_params(axis='x', rotation=90)
#plt.legend(title="Property Complexity")


legend = sns_plot.legend_
sns_plot.axes.legend(loc='upper left')
sns_plot.figure.tight_layout()

sns_plot.figure.savefig(f"{plot_folder}/violinplot_model.png")
sns_plot.figure.savefig(f"{plot_folder}/violinplot_model.pgf")




In [233]:
fig, ax = plt.subplots(figsize=(5, 3)) 
pal = sns.color_palette("magma", 256)


sns_plot = sns.violinplot(data=combined_df, x="property_complexity_category", y="diff", hue="faster", split=True, ax=ax, palette=extreme_colors)
sns_plot.axes.set_xlabel("Property Complexity Category")
sns_plot.axes.set_ylabel("Time Difference in percentage")
sns_plot.axes.tick_params(axis='x', rotation=90)
#plt.legend(title="Property Complexity")


legend = sns_plot.legend_
sns_plot.axes.legend(loc='upper left')
sns_plot.figure.tight_layout()

sns_plot.figure.savefig(f"{plot_folder}/violinplot_prop.png")
sns_plot.figure.savefig(f"{plot_folder}/violinplot_prop.pgf")

In [234]:
fig, ax = plt.subplots(figsize=(5, 3)) 
pal = sns.color_palette("magma", 256)

pos_diff = combined_df[combined_df["faster"]==True]
neg_diff = combined_df[combined_df["faster"]==False]

sns_plot = sns.boxenplot(data=pos_diff, x="model_prop_com_small", y="diff", ax=ax,color=extreme_colors[1])
sns_plot = sns.swarmplot(data=neg_diff, x="model_prop_com_small", y="diff",  dodge=True, ax=ax, color=extreme_colors[0])

sns_plot.axes.set_xlabel("Model / Property Complexity Category")
sns_plot.axes.set_ylabel("Time Difference in percentage")
sns_plot.axes.tick_params(axis='x', rotation=45)
#plt.legend(title="Property Complexity")


legend = sns_plot.legend_
sns_plot.axes.legend(loc='upper left')
sns_plot.figure.tight_layout()

sns_plot.figure.savefig(f"{plot_folder}/boxen_plot.png")
sns_plot.figure.savefig(f"{plot_folder}/boxen_plot.pgf")




In [235]:
g = sns.FacetGrid(combined_df, col="model_prop_com", hue="property_complexity", col_wrap=4, sharex=False, sharey=False)
g.map(sns.histplot, "diff_norm", kde=True)
g.add_legend()

In [236]:
import seaborn as sns
import matplotlib.pyplot as plt



plt.figure(figsize=(14, 6))
sns.violinplot(data=combined_df, x="model_prop_com", y="diff_norm", hue="property_complexity", split=True, inner="quartile")
sns.stripplot(data=combined_df, x="model_prop_com", y="diff_norm", color="black", alpha=0.3, jitter=True, dodge=True)
plt.xticks(rotation=45)
plt.title("Violin + Strip Plot of Normalized Time Difference")
plt.show()

In [237]:
pivot = combined_df.pivot_table(values='diff_norm', index='model_complexity', columns='property_complexity')
sns.heatmap(pivot, annot=True, cmap="coolwarm", center=0)
plt.title("Mean Normalized Time Difference by Complexity")

Text(0.5, 1.0, 'Mean Normalized Time Difference by Complexity')

In [238]:
sns.barplot(data=combined_df, x="model_prop_com", y="diff_norm", ci=95)
plt.xticks(rotation=45)
plt.title("Mean Time Difference by Complexity with 95% CI")

Text(0.5, 1.0, 'Mean Time Difference by Complexity with 95% CI')

In [239]:
import numpy as np
import forestplot as fp



In [240]:
summary_df

,model_prop_com_med,estimate,sem,ll,ul,color,Model Property
0,Model= C / Specification= C,5.252528,3.696260,1.556268,8.948788,green,Model= C / Specification= C
1,Model= C / Specification= H,4.975003,2.106505,2.868498,7.081508,green,Model= C / Specification= H
2,Model= C / Specification= S,3.663627,2.069525,1.594102,5.733152,green,Model= C / Specification= S
3,Model= C / Specification= T,3.413794,1.580068,1.833726,4.993862,green,Model= C / Specification= T
4,Model= H / Specification= C,9.614442,4.054503,5.559939,13.668944,green,Model= H / Specification= C
5,Model= H / Specification= H,9.111401,3.597104,5.514297,12.708505,green,Model= H / Specification= H
6,Model= H / Specification= S,14.514074,10.455442,4.058633,24.969516,green,Model= H / Specification= S
7,Model= H / Specification= T,8.338996,3.743449,4.595547,12.082445,green,Model= H / Specification= T
8,Model= S / Specification= C,5.428918,4.868784,0.560135,10.297702,green,Model= S / Specification= C
9,Model= S / Specification= H,5.276190,2.330663,2.945527,7.606853,green,Model= S / Specification= H


In [241]:
plt.close()

In [ ]:
df_long = pd.melt(
    combined_df,
    id_vars=["model_complexity_category"],
    value_vars=["time_brute_force", "total_time_with_analysis"],
    var_name="method",
    value_name="time"
)

df_long["method"] = df_long["method"].map({
    "time_brute_force": "Brute Force",
    "total_time_with_analysis": "With Analysis"
})

fig, ax = plt.subplots(figsize=(5, 3))
palette = sns.color_palette("Set2", 2)

sns_plot = sns.violinplot(
    data=df_long,
    x="model_complexity_category",
    y="time",
    hue="method",
    split=True,
    ax=ax,
    palette=palette
)

ax.set_xlabel("Model Complexity Category")
ax.set_ylabel("Time (s)")
ax.tick_params(axis='x', rotation=45)
ax.legend(title="Method", loc="upper left")
sns_plot.figure.tight_layout()

sns_plot.figure.savefig(f"{plot_folder}/violinplot_model_split.png")
sns_plot.figure.savefig(f"{plot_folder}/violinplot_model_split.pgf")


In [243]:
combined_df[combined_df["time_brute_force"] ==combined_df["time_brute_force"].max()]

,states,transitions,atomic_props,base_props,refined_props,num_classes,chain_states,bit_width,model_complexity,property_complexity,...,analysis_time,time_mc_refined,diff,model_complexity_category,property_complexity_category,model_prop_com,model_prop_com_small,model_prop_com_med,diff_norm,faster
54,4000,3200,27,4,26,13,18,12,1.55067,1.3574,...,17.702187,502.071333,45.762046,Hard,Standard,Model=Hard / Specification=Standard,H/S,Model= H / Specification= S,457.620461,True


In [247]:
test = combined_df.drop(index=54)
test[test["time_brute_force"] ==test["time_brute_force"].max()]
test = combined_df.drop(index=[12,13,14,15,46,48,50,51,52,54,55,58,74,77,79, 61,63, 67,68,76])

In [248]:
test[test["time_brute_force"] ==test["time_brute_force"].max()]

,states,transitions,atomic_props,base_props,refined_props,num_classes,chain_states,bit_width,model_complexity,property_complexity,...,analysis_time,time_mc_refined,diff,model_complexity_category,property_complexity_category,model_prop_com,model_prop_com_small,model_prop_com_med,diff_norm,faster
71,16000,12800,33,5,36,11,39,9,1.538878,1.932524,...,95.192553,47.409582,1.881378,Hard,Hard,Model=Hard / Specification=Hard,H/H,Model= H / Specification= H,18.813777,True


In [266]:
sns.set_theme(style="darkgrid")
fig, ax = plt.subplots(figsize=(5, 3)) 
df_long = pd.melt(
    test,
    id_vars=["model_complexity_category"],
    value_vars=["time_brute_force", "total_time_with_analysis", "analysis_time"],
    var_name="method",
    value_name="time"
)

df_long["method"] = df_long["method"].map({
    "time_brute_force": "Brute Force",
    "total_time_with_analysis": "With Analysis",
    "analysis_time": "Analysis Time"
})


sns_plot = sns.boxenplot(
    data=df_long,
    x="model_complexity_category",
    y="time",
    hue="method",
    ax=ax,
    dodge=True,
    linewidth=1,
    width=0.6,
    palette =[extreme_colors[0], pal[80], extreme_colors[1]]
)

ax.set_xlabel(None)
ax.set_ylabel("Time (s)")
ax.tick_params(axis='x', rotation=45)
ax.legend(title="Method", loc="upper left")
sns_plot.figure.tight_layout()

sns_plot.figure.savefig(f"{plot_folder}/boxenplot_model_split.png")
sns_plot.figure.savefig(f"{plot_folder}/boxenplot_model_split.pgf")
